In [ ]:
from selenium import webdriver

In [ ]:
import csv
from bs4 import BeautifulSoup

In [ ]:
browser = webdriver.Chrome('/Users/yuyakondo/Desktop/Python web scrapping/chromedriver')

In [ ]:
# access to Amazon.com

#url = 'https://amazon.com'
#browser.get(url)

In [ ]:
def get_url (search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}r&crid=1LX4U1JBSS9JP&sprefix=ultrawide%2Caps%2C235&ref=nb_sb_ss_ts-doa-p_1_9'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [ ]:
url = get_url('ultrawide monitor')
print(url)

In [ ]:
# type in automatically 'ultra wide monitor' in search
browser.get(url)

# Extract the collection

In [ ]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [ ]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [ ]:
len(results)

# prototype record

In [ ]:
item = results[0]

In [ ]:
atag = item.h2.a

In [ ]:
description = atag.text.strip()

In [ ]:
url = 'https://www.amazon.com' + atag.get('href')

In [ ]:
# get the price 

price_parent = item.find('span', 'a-price')

In [ ]:
price_parent.find('span', 'a-offscreen').text

In [ ]:
# grab the rating reviews

ratings = item.i.text

In [ ]:
review_count = item.find('span', {'class': 'a-size-base'}).text

In [ ]:
print(review_count)

# generalized the pattern 

In [ ]:
def extract_record(item):
    
    # description and url 
    
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    # Price 
    
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    # rank and rating
    
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base'}).text
    
    result = (description, price, rating, review_count, url)
    
    return result

In [ ]:
records = [ ]
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    records.append(extract_record(item))

In [ ]:
# see if its work 

records[0]

In [ ]:
# display entire price on the first page

for row in records:
    print(row[1])

# getting the text page 

In [ ]:


def get_url (search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}r&crid=1LX4U1JBSS9JP&sprefix=ultrawide%2Caps%2C235&ref=nb_sb_ss_ts-doa-p_1_9'
    search_term = search_term.replace(' ', '+')
    
    # add term query to url 
    
    url = template.format(search_term)
    
    # add page query placeholder 
    
    url += '&page{}'
    
    return url

# Handling error

In [ ]:
def extract_record(item):
    
    # description and url 
    
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        # Price 
    
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    # rank and rating
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

# Putting it all together

In [ ]:
records = [ ]
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [58]:
import csv
from bs4 import BeautifulSoup

# getting the text page 

def get_url (search_term):
    """Generate a url from search term"""
    template = 'https://www.amazon.com/s?k={}r&crid=1LX4U1JBSS9JP&sprefix=ultrawide%2Caps%2C235&ref=nb_sb_ss_ts-doa-p_1_9'
    search_term = search_term.replace(' ', '+')
    
    # add term query to url 
    
    url = template.format(search_term)
    
    # add page query placeholder 
    
    url += '&page={}'
    
    return url


def extract_record(item):
    
    # description and url 
    
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    try:
        # Price 
    
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    # rank and rating
    
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    
    # start up the webdriver
    
    browser = webdriver.Chrome('/Users/yuyakondo/Desktop/Python web scrapping/chromedriver')
    
    records= [ ]
    url = get_url(search_term)
    
    # grabbing entire page 
    for page in range(1,21):
        browser.get(url.format(page))
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    
    browser.close()
    
    # save data to csv file 
    
    with open('wide_monitor_price_amazon.csv', 'w', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'price', 'rating', 'review_count', 'URL'])
        writer.writerows(records)

In [59]:
# start scrapping

main('Ultra wide monitor')